**React to SwiftUI Code Converter**

Convert your React code into SwiftUI - iOS Native Code.
Compare the results of Frontier Model by OpenAI and Open-Source Model from HuggingFace.

In [1]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [32]:
from google.colab import userdata # גישה ל״סיקרטס״ של ה״נווטבוק״
from openai import OpenAI # גישה לOpenAI
from IPython.display import Markdown, display, update_display # להציג דברים בצורת ״מארק דאון״
from huggingface_hub import login # מאפשר לוגין להאגינגפייס

from transformers import AutoModelForCausalLM, pipeline, AutoTokenizer, BitsAndBytesConfig, TextStreamer, TextIteratorStreamer # שימוש במודל מהאגינג פייס
import torch
import threading

In [3]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [4]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [5]:
react_sample = '''
import React, { useState, useEffect } from 'react';

function Counter() {
  // הגדרת משתנה state לשמירת הערך
  const [count, setCount] = useState(0);

  useEffect(() => {
    console.log(`The count is now: ${count}`);
  }, [count]);

  // פונקציה להגדלת הערך ב-1
  const increment = () => {
    let x= 3
    setCount(count + 1);
  };

  return (
    <div>
      <h1>Counter: {count}</h1>
      <button style={{ backgroundColor: 'blue', padding: 10 }} onClick={increment}>
           Increase
       </button>
    </div>
  );
}

export default Counter;

'''

In [6]:
system_message = "You are an assistant that get code in React language and convert it to its SwiftUI equivalent and Optimized. Do NOT include information - only the new code."
user_prompt = f"{react_sample}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [7]:
def gpt_answer(react_code):
  messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": react_code}
  ]
  response = openai.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages
  )
  return response.choices[0].message.content

In [8]:
def gpt_answer_streaming(react_code):
  messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": react_code}
  ]
  stream = openai.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      stream=True
  )
  res = ""
  for chunk in stream:
    content = chunk.choices[0].delta.content
    if content is not None:
      res += content
      yield res

**Hugging Face Solution**

In [9]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [27]:
checkpoint = "mistralai/Mistral-7B-Instruct-v0.1"
device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

streamer = TextStreamer(tokenizer) # For streaming with HuggingFace
# streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype="auto", quantization_config=quant_config)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [19]:
def convert_react_to_swift_huggingface(react_code):
  is_pipe = False
  if is_pipe:
    prompt = f"""### Instruction:
    Convert the following React code to the optimized SwiftUI equivalent. Do NOT include extra information:

    {react_code}

    ### Response:"""

    result = pipe(prompt, max_new_tokens=200, do_sample=False, temperature=0.7)
    response_text = result[0]['generated_text']
    # חיפוש המיקום של "### Response:" והחזרת החלק אחרי זה
    swiftui_code = response_text.split("### Response:")[-1].strip()
    return "```" + swiftui_code
  else:
    tokenizer_messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": react_code}
    ]
    inputs = tokenizer.apply_chat_template(tokenizer_messages, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens=500)
    response = tokenizer.decode(outputs[0])
    swiftui_code = extract_mistralai_model_response(response)
    return swiftui_code

In [37]:
max_tokens = 2000

def convert_react_to_swift_huggingface_streaming(react_code):
  streaming_massages =  [{"role": "system", "content": system_message}, {"role": "user", "content": react_code}]

  input_ids = tokenizer.apply_chat_template(streaming_massages, return_tensors="pt", add_generation_prompt=True).to("cuda")
  # Initialize an empty string to accumulate the generated result
  result = ""

  for _ in range(max_tokens):
    outputs = model(input_ids)
    next_token_id = outputs.logits[:, -1].argmax(dim=-1).unsqueeze(-1)
    input_ids = torch.cat([input_ids, next_token_id], dim=-1)
    next_token = tokenizer.decode(next_token_id[0], skip_special_tokens=True)
    # Step 8: Accumulate the decoded token into the result string
    result += next_token
    # Step 9: Yield the accumulated result for streaming output
    yield result
    # Step 10: Check if the model predicted the end-of-sequence (EOS) token
    # - tokenizer.eos_token_id: The special token ID representing EOS.
    if next_token_id.item() == tokenizer.eos_token_id:
       break



  # Optimized streaming:
def convert_react_to_swift_huggingface_streaming_optimized(react_code):
  # Step 1: Prepare the messages
  streaming_massages =  [{"role": "system", "content": system_message}, {"role": "user", "content": react_code}]

  # Step 2: Prepare the inputs for the model by applying the chat template
  # The inputs include the conversation history and the user's latest message
  inputs = tokenizer.apply_chat_template(streaming_massages, return_tensors="pt", add_generation_prompt=True).to("cuda")

  # Step 3: Initialize the TextIteratorStreamer
  streamer = TextIteratorStreamer(
      tokenizer,
      skip_prompt=True,  # Ensures that the input prompt is not repeatedly included in the streamed output.
      decode_kwargs={"skip_special_tokens": True}  # Filters out special tokens (e.g., <s>, </s>, <pad>, <cls>, <sep>) from the generated text.
    )

  # Step 4: Create a thread to run the generation process in the background
  thread = threading.Thread(
      target=model.generate,  # Specifies that the model's `generate` method will be run in the thread.
      kwargs={                           # Passes the arguments required for text generation
          "inputs": inputs,              # The tokenized input prompt for the model.
          "max_new_tokens": max_tokens,  # Limits the number of tokens to be generated.
          "streamer": streamer           # The TextIteratorStreamer to handle streaming the output.
            }
   )

  # Step 5: Start the thread to begin the generation process
  thread.start()

  # Step 6: Initialize an empty string to accumulate the growing output
  accumulated_reply = ""

  # Step 7: Stream the output progressively
  for text_chunk in streamer:  # Iterate over each chunk of text streamed by the model
     # Filter out any unexpected special tokens manually if they appear to ensure a clean output
     # `<|eot_id|>` is a special token (e.g., end-of-text marker) that may still appear in some outputs
    filtered_chunk = text_chunk.replace("<|eot_id|>", "").replace("</s>", "")

    # Append the filtered chunk to the accumulated text that holds all the generated text seen so far
    accumulated_reply += filtered_chunk

    # Yield the accumulated text to the calling function/UI for progressive updates,
    # ensuring the output is continuously refreshed with new content
    yield accumulated_reply

**UI Gradio**

In [13]:
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 118.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.3 MB/s eta 0:00:00


In [14]:
def handle_submit(react_code, model_type):
  if model_type == "mistralai":
    return convert_react_to_swift_huggingface(react_code)
  else:
    return gpt_answer(react_code)


In [33]:
def handle_submit_streaming(react_code, model_type):
  if model_type == "mistralai":
    for chunk in convert_react_to_swift_huggingface_streaming_optimized(react_code):
      yield chunk
  else:
    for chunk in gpt_answer_streaming(react_code):
      yield chunk


In [30]:
def extract_mistralai_model_response(full_response: str) -> str:
    # מחפש את התחלה אחרי הטאג [/INST]
    start = full_response.find('[/INST]')
    if start != -1:
        start += len('[/INST]')
    else:
        start = 0

    # חותך עד הטאג הסוגר </s>
    end = full_response.find('</s>', start)
    if end == -1:
        end = len(full_response)

    # מחזיר את הטקסט הרלוונטי ומנקה רווחים
    return "```" + full_response[start:end].strip()

In [17]:
custom_css = """
/* כללי */
body {
    background-color: #003f9c;
    font-family: 'Segoe UI', sans-serif;
}



/* כפתור */
button {
    background-color: #4CAF50 !important;
    color: white !important;
    font-weight: bold;
    border-radius: 8px;
    padding: 10px 16px;
    transition: background-color 0.3s;
}

button:hover {
    background-color: #004f70 !important;
}

/* Markdown (קוד Swift) */
.markdown-body pre code {
    background-color: #f0f0f0;
    color: #2e2e2e;
    padding: 16px;
    border-radius: 10px;
    font-size: 14px;
    display: block;
    overflow-x: auto;
}

/* דרופדאון */
select {
    padding: 8px;
    border-radius: 8px;
    font-size: 14px;
}
"""

In [38]:
with gr.Blocks(css=custom_css) as demo:
    with gr.Row():
        react_code_input = gr.Textbox(label="React Code", value=react_sample, lines=14, elem_classes=["input"])
        output_markdown = gr.Markdown() #gr.Textbox(label="SwiftUI Code", lines=14) #

    with gr.Row():
        model_dropdown = gr.Dropdown(["mistralai", "gpt-4o-mini"], label="בחר את המודל", value="gpt-4o-mini")

    with gr.Row():
        submit_btn = gr.Button("Generate SwiftUI Code!")

    submit_btn.click(
        fn=handle_submit_streaming,
        inputs=[react_code_input, model_dropdown],
        outputs=output_markdown
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://64c51d34dbaaf627ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
